# Polyglot API Demonstration

SurrealEngine is designed to be **Polyglot**, meaning the same Document models can be used in both **Synchronous** (blocking) and **Asynchronous** (non-blocking) applications seamlessly.

The behavior is determined entirely by the **Connection** you use.

- `create_connection(..., async_mode=False)` -> Methods block and return results directly.
- `create_connection(..., async_mode=True)` -> Methods return awaitables (Futures).

This notebook demonstrates both patterns using the same `User` model.

In [1]:
from surrealengine import Document, StringField, create_connection, ConnectionRegistry
import asyncio

# Define a shared model
class User(Document):
    name = StringField()
    role = StringField()
    
    class Meta:
        collection = "poly_users"

## 1. Synchronous Mode (Blocking)

Ideal for scripts, CLI tools, or traditional web frameworks like Flask/Django.

In [2]:
print("--- SYNCHRONOUS MODE ---")

# 1. Create a SYNC connection
sync_conn = create_connection(
    url="ws://db:8000/rpc",
    username="root",
    password="root",
    namespace="demo",
    database="polyglot",
    async_mode=False,  # <--- EXACTLY THIS
    make_default=True
)
sync_conn.connect() # Blocks until connected
print("Connected synchronously.")

# 2. Create Table (Blocking)
User.create_table()
print("Table created.")

# 3. Save User (Blocking)
user1 = User(name="Alice Sync", role="admin")
user1.save() # No await!
print(f"Saved user: {user1.name} with ID: {user1.id}")

# 4. Query (Blocking)
users = User.objects.all()
print(f"Found {len(users)} users synchronously.")

sync_conn.disconnect()

--- SYNCHRONOUS MODE ---
Connected synchronously.
Table created.
Saved user: Alice Sync with ID: poly_users:7mzo7xhy7jzwohppbdmr
Found 3 users synchronously.


## 2. Asynchronous Mode (Non-Blocking)

Ideal for high-performance applications using FastAPI, Quart, or asyncio.

In [ ]:
print("\n--- ASYNCHRONOUS MODE ---")

async def async_demo():
    # 1. Create an ASYNC connection
    async_conn = create_connection(
        url="ws://db:8000/rpc",
        username="root",
        password="root",
        namespace="demo",
        database="polyglot",
        async_mode=True,   # <--- EXACTLY THIS
        make_default=True  # Swaps the default connection to this one
    )
    await async_conn.connect()
    print("Connected asynchronously.")

    # 2. Save User (Non-Blocking)
    user2 = User(name="Bob Async", role="user")
    await user2.save() # Must await!
    print(f"Saved user: {user2.name} with ID: {user2.id}")

    # 3. Query (Non-Blocking)
    users = await User.objects.all()
    print(f"Found {len(users)} users asynchronously.")
    
    await async_conn.disconnect()

# Run the async loop
await async_demo()